In [1]:
import numpy as np
import time
import qcodes
import pyvisa as visa
visa.log_to_screen()

from qcodes.instrument_drivers.yokogawa.GS200 import GS200

In [2]:
visa.ResourceManager().list_resources()

2022-06-13 14:26:04,294 - pyvisa - DEBUG - No visa library specified, trying to find alternatives.
2022-06-13 14:26:04,299 - pyvisa - DEBUG - Environment variable PYVISA_LIBRARY is unset.
2022-06-13 14:26:04,310 - pyvisa - DEBUG - Not loading dll_extra_paths because we are not on Windows or Python < 3.8
2022-06-13 14:26:04,467 - pyvisa - DEBUG - Automatically found library files: ['/usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0', None, None, None, None]
2022-06-13 14:26:04,468 - pyvisa - DEBUG - No user defined library files
2022-06-13 14:26:04,469 - pyvisa - DEBUG - The IVI implementation available
2022-06-13 14:26:04,469 - pyvisa - DEBUG - Not loading dll_extra_paths because we are not on Windows or Python < 3.8
2022-06-13 14:26:04,561 - pyvisa - DEBUG - Automatically found library files: ['/usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0', None, None, None, None]
2022-06-13 14:26:04,564 - pyvisa - DEBUG - No user defined library files
2022-06-13 14:26:04,566 - pyvisa - DEBUG - Library signa

('ASRL5::INSTR',)

In [3]:
# intialize the instrument with the following parameters
gs = GS200("gs200", address='USB0::0xB21::0x39::91LA25023::INSTR', terminator="\n")
#gs = GS200("gs200", address='ASRL5::INSTR', terminator="\n")

2022-06-13 14:26:05,939 - pyvisa - DEBUG - No visa library specified, trying to find alternatives.
2022-06-13 14:26:05,943 - pyvisa - DEBUG - Environment variable PYVISA_LIBRARY is unset.
2022-06-13 14:26:05,946 - pyvisa - DEBUG - Not loading dll_extra_paths because we are not on Windows or Python < 3.8
2022-06-13 14:26:06,060 - pyvisa - DEBUG - Automatically found library files: ['/usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0', None, None, None, None]
2022-06-13 14:26:06,061 - pyvisa - DEBUG - No user defined library files
2022-06-13 14:26:06,063 - pyvisa - DEBUG - The IVI implementation available
2022-06-13 14:26:06,064 - pyvisa - DEBUG - Not loading dll_extra_paths because we are not on Windows or Python < 3.8
2022-06-13 14:26:06,162 - pyvisa - DEBUG - Automatically found library files: ['/usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0', None, None, None, None]
2022-06-13 14:26:06,163 - pyvisa - DEBUG - No user defined library files
2022-06-13 14:26:06,164 - pyvisa - DEBUG - Reusing Resou

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [ ]:
gs.source_mode()

In [ ]:
gs.source_mode('CURR')
gs.source_mode()

In [ ]:
gs.output()

In [ ]:
gs.current(0.005)
